In [1]:
import os
import time
import copy
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from nets.yolo4 import YoloBody
from nets.yolo_training import YOLOLoss, Generator

In [2]:
#---------------------------------------------------#
#   获得类和先验框
#---------------------------------------------------#
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names


def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape([-1,3,2])[::-1,:,:]


#---------------------------------------------------#
#   训练一个epoch
#---------------------------------------------------#
def fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen,genval, Epoch, cuda, optimizer, lr_scheduler):
    total_loss = 0
    val_loss = 0
    print('\n' + '-' * 10 + 'Train one epoch.' + '-' * 10)
    print('Epoch:'+ str(epoch+1) + '/' + str(Epoch))
    print('Start Training.')
    net.train()
    for iteration in range(epoch_size):
        start_time = time.time()
        images, targets = next(gen)
        with torch.no_grad():
            if cuda:
                images = Variable(torch.from_numpy(images).type(torch.FloatTensor)).cuda()
                targets = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets]
            else:
                images = Variable(torch.from_numpy(images).type(torch.FloatTensor))
                targets = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets]
        optimizer.zero_grad()
        outputs = net(images)
        losses = []
        for i in range(3):
            loss_item = yolo_losses[i](outputs[i], targets)
            losses.append(loss_item[0])
        loss = sum(losses)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss
        waste_time = time.time() - start_time
        if iteration == 0 or (iteration+1) % 10 == 0:
            print('step:' + str(iteration+1) + '/' + str(epoch_size) + ' || Total Loss: %.4f || %.4fs/step' % (total_loss/(iteration+1), waste_time))
    print('Finish Training.')
    '''        
    print('Start Validation.')
    net.eval()
    for iteration in range(epoch_size_val):
        images_val, targets_val = next(genval)

        with torch.no_grad():
            if cuda:
                images_val = Variable(torch.from_numpy(images_val).type(torch.FloatTensor)).cuda()
                targets_val = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets_val]
            else:
                images_val = Variable(torch.from_numpy(images_val).type(torch.FloatTensor))
                targets_val = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets_val]
            optimizer.zero_grad()
            outputs = net(images_val)
            losses = []
            for i in range(3):
                loss_item = yolo_losses[i](outputs[i], targets_val)
                losses.append(loss_item[0])
            loss = sum(losses)
            val_loss += loss
    print('Finish Validation')
    '''
    print('Total Loss: %.4f || Val Loss: %.4f ' % (total_loss/(epoch_size+1), val_loss/(epoch_size_val+1)))
    
    return total_loss/(epoch_size+1), val_loss/(epoch_size_val+1)

In [3]:
#-------------------------------#
#   输入的shape大小
#   显存比较小可以使用416x416
#   显存比较大可以使用608x608
#-------------------------------#
#input_shape = (416,416)
input_shape = (608, 608)

#-------------------------------#
#   tricks的使用设置
#-------------------------------#
Cosine_lr = True
mosaic = True
# 用于设定是否使用cuda
Cuda = True
smoooth_label = 0.03

#-------------------------------#
#   获得训练集和验证集的annotations
#   
#-------------------------------#
train_annotation_path = 'model_data/mask_train.txt'
val_annotation_path = 'model_data/mask_val.txt'

#-------------------------------#
#   获得先验框和类
#-------------------------------#
anchors_path = 'model_data/yolo_anchors.txt'
classes_path = 'model_data/mask_classes.txt'   
class_names = get_classes(classes_path)
anchors = get_anchors(anchors_path)
num_classes = len(class_names)

In [6]:
# 创建模型
model = YoloBody(len(anchors[0]), num_classes)
#model_path = "model_data/yolov4_coco_pretrained_weights.pth"
model_path = "model_data/yolov4_maskdetect_weights0.pth"
# 加快模型训练的效率
print('Loading pretrained model weights.')
model_dict = model.state_dict()
pretrained_dict = torch.load(model_path)
pretrained_dict = {k: v for k, v in pretrained_dict.items() if np.shape(model_dict[k]) ==  np.shape(v)}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
print('Finished!')

if Cuda:
    net = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    net = net.cuda()

# 建立loss函数
yolo_losses = []
for i in range(3):
    yolo_losses.append(YOLOLoss(np.reshape(anchors, [-1,2]), num_classes, \
                                (input_shape[1], input_shape[0]), smoooth_label, Cuda))
# read train lines and val lines
with open(train_annotation_path) as f:
    train_lines = f.readlines()
with open(val_annotation_path) as f:
    val_lines = f.readlines()
num_train = len(train_lines)
num_val = len(val_lines)

Loading pretrained model weights.
Finished!


In [6]:
#------------------------------------#
#   先冻结backbone训练
#------------------------------------#
lr = 1e-3
Batch_size = 4
Init_Epoch = 0
Freeze_Epoch = 25
        
optimizer = optim.Adam(net.parameters(), lr, weight_decay=5e-4)
if Cosine_lr:
    lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-5)
else:
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

gen = Generator(Batch_size, train_lines, (input_shape[0], input_shape[1])).generate(mosaic = mosaic)
gen_val = Generator(Batch_size, val_lines, (input_shape[0], input_shape[1])).generate(mosaic = False)
                        
epoch_size = int(max(1, num_train//Batch_size//2.5)) if mosaic else max(1, num_train//Batch_size)
epoch_size_val = num_val//Batch_size
for param in model.backbone.parameters():
    param.requires_grad = False

best_loss = 99999999.0
best_model_weights = copy.deepcopy(net.state_dict())
for epoch in range(Init_Epoch, Freeze_Epoch):
    total_loss, val_loss = fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen, gen_val, 
                                         Freeze_Epoch, Cuda, optimizer, lr_scheduler)
    if total_loss < best_loss:
        best_loss = total_loss
        best_model_weights = copy.deepcopy(model.state_dict())
    with open('total_loss.csv', mode='a+') as total_loss_file:
        total_loss_file.write(str(total_loss.item()) + '\n')
    #with open('val_loss.csv', mode='a+') as val_loss_file:
    #    val_loss_file.write(str(val_loss.item()) + '\n')
torch.save(best_model_weights, 'model_data/yolov4_maskdetect_weights0.pth')


----------Train one epoch.----------
Epoch:1/25
Start Training.
step:1/120 || Total Loss: 15688.8691 || 6.8430s/step
step:10/120 || Total Loss: 12204.3193 || 1.6851s/step
step:20/120 || Total Loss: 9636.8506 || 1.4570s/step
step:30/120 || Total Loss: 7773.7681 || 1.6354s/step
step:40/120 || Total Loss: 6444.1396 || 1.7304s/step
step:50/120 || Total Loss: 5482.1118 || 1.5075s/step
step:60/120 || Total Loss: 4764.4355 || 1.3454s/step
step:70/120 || Total Loss: 4211.4189 || 1.4012s/step
step:80/120 || Total Loss: 3774.4348 || 1.7187s/step
step:90/120 || Total Loss: 3421.0762 || 1.5336s/step
step:100/120 || Total Loss: 3128.3455 || 1.4144s/step


/root/miniconda3/envs/myconda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


step:110/120 || Total Loss: 2882.8147 || 1.3798s/step
step:120/120 || Total Loss: 2674.7305 || 1.4065s/step
Finish Training.
Total Loss: 2652.6250 || Val Loss: 0.0000 

----------Train one epoch.----------
Epoch:2/25
Start Training.
step:1/120 || Total Loss: 359.2517 || 1.8192s/step
step:10/120 || Total Loss: 345.9673 || 1.3860s/step
step:20/120 || Total Loss: 328.0730 || 2.3537s/step
step:30/120 || Total Loss: 319.5523 || 1.9720s/step
step:40/120 || Total Loss: 301.3265 || 1.2407s/step
step:50/120 || Total Loss: 288.6107 || 1.5783s/step
step:60/120 || Total Loss: 278.3066 || 1.6308s/step
step:70/120 || Total Loss: 267.8594 || 1.6753s/step
step:80/120 || Total Loss: 257.0878 || 1.4918s/step
step:90/120 || Total Loss: 248.1550 || 1.1582s/step
step:100/120 || Total Loss: 239.4298 || 1.2975s/step
step:110/120 || Total Loss: 235.7772 || 1.4822s/step
step:120/120 || Total Loss: 228.1834 || 1.4821s/step
Finish Training.
Total Loss: 226.2975 || Val Loss: 0.0000 

----------Train one epoch.---

In [ ]:
#------------------------------------#
#   解冻backbone后训练
#------------------------------------#
lr = 1e-4
Batch_size = 2
Freeze_Epoch = 25
Unfreeze_Epoch = 50

optimizer = optim.Adam(net.parameters(), lr, weight_decay=5e-4)
if Cosine_lr:
    lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-5)
else:
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

gen = Generator(Batch_size, train_lines, (input_shape[0], input_shape[1])).generate(mosaic = mosaic)
gen_val = Generator(Batch_size, val_lines, (input_shape[0], input_shape[1])).generate(mosaic = False)
                        
epoch_size = int(max(1, num_train//Batch_size//2.5)) if mosaic else max(1, num_train//Batch_size)
epoch_size_val = num_val//Batch_size
for param in model.backbone.parameters():
    param.requires_grad = True

for epoch in range(Freeze_Epoch, Unfreeze_Epoch):
    total_loss, val_loss = fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen, gen_val, 
                                         Unfreeze_Epoch, Cuda, optimizer, lr_scheduler)
    if total_loss < best_loss:
        best_loss = total_loss
        best_model_weights = copy.deepcopy(model.state_dict())
    with open('total_loss.csv', mode='a+') as total_loss_file:
        total_loss_file.write(str(total_loss.item()) + '\n')
    #with open('val_loss.csv', mode='a+') as val_loss_file:
    #    val_loss_file.write(str(val_loss.item() + '\n')
torch.save(best_model_weights, 'model_data/yolov4_maskdetect_weights1.pth')


----------Train one epoch.----------
Epoch:26/50
Start Training.
step:1/240 || Total Loss: 20.0526 || 1.0939s/step
step:10/240 || Total Loss: 15.7005 || 0.8445s/step
step:20/240 || Total Loss: 14.3441 || 0.8856s/step
step:30/240 || Total Loss: 13.1000 || 0.9173s/step
step:40/240 || Total Loss: 17.5097 || 0.9113s/step
step:50/240 || Total Loss: 15.8164 || 0.8353s/step
step:60/240 || Total Loss: 16.0407 || 0.8812s/step
step:70/240 || Total Loss: 16.0285 || 0.7731s/step
step:80/240 || Total Loss: 15.3731 || 0.7119s/step
step:90/240 || Total Loss: 14.8034 || 0.9056s/step
step:100/240 || Total Loss: 14.7215 || 0.7492s/step
step:110/240 || Total Loss: 18.2945 || 1.4890s/step
step:120/240 || Total Loss: 19.0468 || 1.0130s/step
step:130/240 || Total Loss: 18.3268 || 0.9163s/step
step:140/240 || Total Loss: 17.4725 || 0.8013s/step
step:150/240 || Total Loss: 16.9751 || 0.9707s/step
step:160/240 || Total Loss: 18.9430 || 1.2700s/step
step:170/240 || Total Loss: 19.3538 || 0.7483s/step
step:180/